### Ragas is a framework that helps you evaluate your Retrieval Augmented Generation (RAG) pipelines


In [6]:
#!pip install openai==0.28.1
#!pip install openai --upgrade
#!pip install ragas
#!pip install unstructured
#!pip install langchain[all]
#!pip install --upgrade langchain

#!pip install playwright
#!pip install -U selenium unstructured
#!pip install --upgrade langchain langchain-community langchainhub langchain-openai langchain-chroma bs4

In [1]:
#!pip install pydantic==2.5
#!pip install rapidocr-onnxruntime

In [1]:
import os, json
from langchain.chains import AnalyzeDocumentChain
from langchain_openai import ChatOpenAI

from utils import OPENAI_API_KEY

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY 
from llm_utils import *
#os.environ["LANGCHAIN_TRACING_V2"] = "true"

#openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
#from langchain_community.document_loaders import SeleniumURLLoader,  DirectoryLoader, PyPDFLoader
#from langchain.text_splitter import CharacterTextSplitter,  RecursiveCharacterTextSplitter

In [3]:
from langchain_openai import ChatOpenAI
#llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)  

In [4]:
#loader_url =SeleniumURLLoader(["https://arxiv.org/pdf/2312.10997",
pdf_list = ["./data/data_rag/s41524-023-01062-z.pdf",
    "./data/data_rag/s41699-023-00369-1.pdf"
         #   "https://github.com/HSE-LAMBDA/ai4material_design/tree/main/docs/CONSTRUCTOR-MOCK.md"
         #   "https://github.com/HSE-LAMBDA/ai4material_design/blob/main/docs/CONSTRUCTOR.md",
         #   "https://github.com/HSE-LAMBDA/ai4material_design/blob/main/docs/DATA.md",
         #   "https://github.com/HSE-LAMBDA/ai4material_design/blob/main/docs/ENVIRONMENT.md",
         #   "https://github.com/HSE-LAMBDA/ai4material_design/blob/main/docs/GENERATING-CONSTRUCTOR.md",
         #   "https://github.com/HSE-LAMBDA/ai4material_design/blob/main/docs/GENERATING-MOCK.md",
         #   "https://github.com/HSE-LAMBDA/ai4material_design/blob/main/docs/PILOT.md",
         #   "https://github.com/HSE-LAMBDA/ai4material_design/blob/main/docs/SPARSE-PAPER.md"
          #  "https://www.nature.com/articles/s41377-024-01407-3",
          #  "https://www.nature.com/articles/s41565-023-01407-1",
          #  "https://www.nature.com/articles/s41699-023-00369-1",
           ]
                               
chunks, documents = load_pdf_documets(pdf_list, tokens=5000)
print(len(chunks), len(documents))

35 35


In [5]:
from langchain.chains.question_answering import load_qa_chain

qa_chain = load_qa_chain(llm, chain_type="map_reduce")

#qa_chain = load_qa_chain(llm, chain_type="map_rerank")

In [141]:
qa_document_chain = AnalyzeDocumentChain(combine_docs_chain=qa_chain, 
                                         text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000, 
                                                                                      chunk_overlap=10))



In [185]:
qa_document_chain = AnalyzeDocumentChain(combine_docs_chain=qa_chain, 
                                         text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000, 
                                                                                      chunk_overlap=10))


nodes = set()

for i,doc in enumerate(documents):
    if not len(nodes):
        answer = qa_document_chain.run(
                 input_document=doc.page_content,
                 question=("""
                 You are an expert in analyzing scientific articles.
                 You are also an expert in text mining and graph generation,
                 which extracts entities or key concepts and their relationships from given documents,
                 and formats them into a connected graph consisting of entities as nodes 
                 and their relationships as edges.
                 Your task is to carefully analyze the given text and extract the main related entities/nodes.
                 These entities/nodes should represent the key concepts of the text.
                 Also you need remember the relationships between these entities.

                 Format the output as a list of node names and their shot descriptions without numbers,
                 splited by \n.
                 Example: node_name1 : short_description1 \n
                           node_name2 : short_description2 \n
                           ...
                 """   
                )
              )
    else: 
        #_nodes = " ,".join([s.split(":")[0].strip() for s in nodes if len(s)])
        llm_nodes = llm.invoke(f"""
                    You will get a list of concepts with descriptions:
                    {' ,'.join(list(nodes))}
                    Your task is to
                    analyze and combine those that are paraphrases or near-duplicates of each other
                    and make a new list without repetitions.
                    Output a list of concepts with descriptions in the same format in which you received them, 
                    without numbers.
                    """)
        _nodes = [s.strip() for s in llm_nodes.content.split("\n") if len(s)]
        
        answer = qa_document_chain.run(
                 input_document=doc.page_content,
                 question=(f"""
                 You are an expert in analyzing scientific articles.
                 You are also an expert in text mining and graph generation,
                 which extracts entities or key concepts and their relationships from given documents,
                 and formats them into a connected graph consisting of entities as nodes 
                 and their relationships as edges.
                 You are given a list of entities: {' ,'.join(_nodes)}
                 Your task is to carefully analyze the given text and extract the main related entities/nodes.
                 Be careful not to add entities/nodes if they are similar to an existing ones.
                 These entities/nodes should represent the key concepts of the text.
    
                 Format the output as a list of node names and their shot descriptions without numbers,
                 splited by \n.
                 Example: node_name1 : short_description1 \n
                           node_name2 : short_description2 \n
                           ...
                
                 
                 """
                 )
               )
    nodes = nodes.union(set(s.strip() for s in answer.split("\n")))
    print(f"Document {i}, summary number of entities {len(nodes)}")
         

llm_nodes = llm.invoke(f"""
                    You will get a list of concepts with descriptions:
                    {' ,'.join(list(nodes))}
                    Your task is to
                    analyze and combine those that are paraphrases or near-duplicates of each other
                    and make a new list without repetitions.
                    Output a list of concepts with descriptions in the same format in which you received them, 
                    without numbers.
                    """)
nodes  = [s.strip() for s in llm_nodes.content.split("\n") if len(s)]
nodes

Document 0, summary number of entities 35
Document 1, summary number of entities 40
Document 2, summary number of entities 41
Document 3, summary number of entities 41
Document 4, summary number of entities 41
Document 5, summary number of entities 41
Document 6, summary number of entities 41
Document 7, summary number of entities 41
Document 8, summary number of entities 41
Document 9, summary number of entities 41
Document 10, summary number of entities 41
Document 11, summary number of entities 41
Document 12, summary number of entities 41
Document 13, summary number of entities 41
Document 14, summary number of entities 41
Document 15, summary number of entities 41
Document 16, summary number of entities 41
Document 17, summary number of entities 41
Document 18, summary number of entities 41
Document 19, summary number of entities 41
Document 20, summary number of entities 41
Document 21, summary number of entities 41
Document 22, summary number of entities 41
Document 23, summary 

['density functional theory : quantum mechanical method used for calculating the electronic structure of many-body systems',
 'crystalline materials : materials with a structured arrangement of atoms, which can be modified through defect engineering',
 'defects : imperfections in 2D materials that can be manipulated to tune properties',
 'energy prediction error : measure of the accuracy of energy predictions, with a reported drop of at least 3.7 times in this methodology',
 'batteries : application for which materials are identified using machine learning methods',
 'defect configuration : arrangement of defects in 2D materials that affects their characteristics',
 'CGCNN : graph neural network architecture used for atomistic predictions',
 'quantum computing : advanced computing technology that utilizes quantum mechanics',
 'computational costs : resources required to perform calculations, significantly lower with machine learning methods compared to traditional methods',
 'machine l

In [243]:
def get_labels(a_list):
    edges = set()
    for e in a_list:
        e = [s.strip().strip("'").strip('"') for s in extract_main_parentheses(e).split(',')]
        try:
            edges.add(e[2])
        except:
            #print(e)
            pass
    return edges

In [247]:
qa_document_chain = AnalyzeDocumentChain(combine_docs_chain=qa_chain, 
                                         text_splitter=RecursiveCharacterTextSplitter(chunk_size=5000, 
                                                                                      chunk_overlap=500))


answer_list = []
labels = get_labels(answer_list)
labels = " ,".join(list(labels))


for i, doc in enumerate(documents):
    answer = qa_document_chain.run(
    input_document=doc.page_content,
    question=(f"""
You are an expert in analyzing scientific articles.\n
                 You are also an expert in text mining and graph generation,
                 which extracts entities or key concepts and their relationships from given documents,
                 and formats them into a connected graph consisting of entities as nodes 
                 and their relationships as edges.\n\n
Your task is to analyze the given text and given list of nodes: \n
{[' ,'.join(list(nodes))]} \n
and extract relationships between these nodes, these relationships have a direction.\n\n
Each  node has to have at least one relationship with anoter nodes  
but  can have multiple relationships with one or other nodes.
Try not to create new relationship names if the list:\n
{[labels]} \n
contains paraphrases or near-duplicates.\n\n

Format the output as a list of tuples. 
Each tuple should contain 3 elements : a source node, a target node, a relationship type.
"""
           )
        )
    
    answer_list += [s.strip('-').strip() for s in  answer.split('\n\n')[-1].split('\n')]
    print(f"Document {i}, summary number of edges {len(answer_list)}")
    
    labels = get_labels(answer_list)
    labels = " ,".join(list(labels))
    #print(answer)

Document 0, summary number of edges 48
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
Document 1, summary number of edges 94
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
Document 2, summary number of edges 122
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
Document 3, summary number of edges 152
['No matching parentheses 

Document 12, summary number of edges 461
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No ma

Document 18, summary number of edges 675
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No ma

Document 22, summary number of edges 823
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No ma

Document 26, summary number of edges 952
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No ma

Document 29, summary number of edges 1058
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No m

Document 32, summary number of edges 1162
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No m

In [187]:
def extract_main_parentheses(text):
    start = -1  # To track the position of the first opening parenthesis
    end = -1    # To track the position of the closing parenthesis
    counter = 0 # To count the parentheses

    for i, char in enumerate(text):
        if char == '(':
            if counter == 0:
                start = i  # Mark the position of the first opening parenthesis
            counter += 1
        elif char == ')':
            counter -= 1
            if counter == 0:
                end = i  # Mark the position of the matching closing parenthesis
                break

    # Extract and return the substring between the outermost parentheses
    if start != -1 and end != -1:
        return text[start+1:end]
    else:
        return "No matching parentheses found"

# Example usage
text = answer_list[131]
result = extract_main_parentheses(text)
print(f"Extracted substring: '{result}'")


Extracted substring: 'materials databases, machine learning, "facilitate"'


In [248]:
import networkx as nx

def create_graph(nodes, edges):
    """
    make a graph from list of nodes
    and list of tuples each tuple has a source node, a target node and a relationship name
    """
    
    G = nx.DiGraph()
    
    if nodes:
        for n in nodes:
            n = n.split(':')
            G.add_node(n[0].strip('-').strip(), description=n[1].strip('.').strip())
    if edges:
        for e in edges:
            e = [s.strip().strip("'").strip('"') for s in extract_main_parentheses(e).split(',')]
            try:
                G.add_node(e[0])
                G.add_node(e[1])
                G.add_edge(e[0], e[1], name=e[2])
            except: 
                print(e)
        
    return G


In [249]:
G = create_graph(nodes, answer_list)

['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching parentheses found']
['No matching 

In [250]:

G.nodes, len(G.nodes)

(NodeView(('density functional theory', 'crystalline materials', 'defects', 'energy prediction error', 'batteries', 'defect configuration', 'CGCNN', 'quantum computing', 'computational costs', 'machine learning', 'lattices', 'quantum phenomena', 'electronic structures', 'GemNet', 'neural network', 'atomic-scale tailoring', 'catalysts', 'lattice structure', 'point defects', 'complexes of multiple point defects', 'two-dimensional materials', 'two-dimensional atomic layers', 'resource-efficiency', 'structure-property relation', 'MEGNet', 'SchNet', 'defect engineering', 'photovoltaics', 'ab-initio calculations', 'graph neural networks', 'exchange interaction', 'materials databases', 'energetic structures', 'No matching parentheses found', 'properties', 'methodology', 'characteristics', 'atomistic predictions', 'properties of 2D materials', 'material properties', 'configuration of 2D materials', 'new properties', 'chemical reactions', 'high technology applications', 'exfoliation methods', '

In [257]:
print("Number of edges:",len(G.edges))



# Function to display all edges with their names and attributes
def print_graph_edges(G):
    edge_names = set()
    for edge in G.edges(data=True):
        print(f"Edge: {edge[0]} -> {edge[2].get('name', 'No name')} -> {edge[1]}")
        edge_names.add(edge[2].get('name', 'No name'))
    print("Unique edge names:", len(edge_names))
    return edge_names
# Print all edges with their attributes
edge_names = print_graph_edges(G)
print("Edges labels:",len(labels.split(",")))

Number of edges: 134
Edge: density functional theory -> is a type of -> ab-initio calculations
Edge: density functional theory -> is a quantum mechanical method used for calculating -> electronic structures
Edge: crystalline materials -> can be modified through -> defect engineering
Edge: defects -> can affect -> properties
Edge: defects -> are particularly relevant in the context of -> quantum phenomena
Edge: defects -> can be manipulated to tune -> two-dimensional materials
Edge: defects -> affects -> defect configuration
Edge: defects -> describe the behavior of -> electronic structures
Edge: defects -> is a specific type of -> defect engineering
Edge: defects -> identified using -> batteries
Edge: defects -> technology that often uses -> photovoltaics
Edge: defects -> involves modifying -> atomic-scale tailoring
Edge: defects -> can lead to complex behaviors and -> complexes of multiple point defects
Edge: defects -> affects -> energy prediction error
Edge: defects -> is a specific

In [258]:
# Save the graph as GEXF including edge attributes
nx.write_gexf(G, "graph_ai4mat_articles_v2.gexf")

print("Graph saved as graph_ai4mat_articles_v2.gexf")

Graph saved as graph_ai4mat_articles_v2.gexf


In [ ]:
G.pl

In [ ]:
# Merge the two graphs
#G_xml = nx.compose(G_xml, create_graph_from_xml(xml_list[1]))

In [ ]:
print(answer)

### Topics of documents

In [52]:
list_dominan = []

first = True
for doc in documents:
    if  first:
        answer = qa_document_chain.run(
            input_document=doc.page_content,
            question=(f""""
                 You are an expert in analyzing scientific papers.
                 Read  a text carefully and answer of the question:
                 What are the five subject areas of research and the text study?
                 Compare these five subjects with the existing ones in the list:\n
                 {topic_list}\n
                 and add new topics to the list, \n
                 but avoid paraphrasing topics or topics with almost duplicating meaning.
                 """)
            )
   #     first = False
   # else:
   #     answer = qa_document_chain.run(
   #         input_document=doc.page_content,
   #         question=(f""""
   #              You are an expert in analyzing scientific papers.
   #              Read a text carefully and answer of the question:
   #              What are the five subject areas of research and the text study?
   #              """)
   #         )
   # if len(list_dominan) > 10:
   #     answer = qa_document_chain.run(
   #         input_document=" ".join(list_dominan),
   #         question=(f""""
   #              You are an expert in analyzing scientific papers.
   #              Create  10 main areas of research that study contect of the text.
   #              """)
   #         )
    print(doc.metadata)
    print(answer)
    print()
    list_dominan += [w[2:] for w in answer.split("\n") if  len(w) and w[0].isdigit()]

{'source': './data/data_rag/s41524-023-01062-z.pdf', 'page': 0}
The relevant text mentions the following subject areas of research:

1. Two-dimensional materials
2. (Opto-) electronic devices
3. High technology applications
4. Properties of defects in 2D materials
5. Machine learning approaches for material property estimation

Additionally, other areas mentioned include:

1. Energy prediction error in methodologies
2. Resource efficiency in training and inference
3. Atomic-scale tailoring of materials
4. Defect engineering in crystalline materials
5. Two-dimensional atomic layers and their properties

Other areas also include:

1. Semiconductor industry
2. Quantum computing
3. Catalysts
4. Photovoltaics
5. Structure-property relation for defects in crystals

And further areas include:

1. Defect prediction in materials
2. Chemical and configuration space exploration
3. Deep learning methods for atomistic predictions
4. Machine learning methods trained on density functional theory (DFT

In [58]:
print(" \n".join(list_dominan))

 Two-dimensional materials 
 (Opto-) electronic devices 
 High technology applications 
 Properties of defects in 2D materials 
 Machine learning approaches for material property estimation 
 Energy prediction error in methodologies 
 Resource efficiency in training and inference 
 Atomic-scale tailoring of materials 
 Defect engineering in crystalline materials 
 Two-dimensional atomic layers and their properties 
 Semiconductor industry 
 Quantum computing 
 Catalysts 
 Photovoltaics 
 Structure-property relation for defects in crystals 
 Defect prediction in materials 
 Chemical and configuration space exploration 
 Deep learning methods for atomistic predictions 
 Machine learning methods trained on density functional theory (DFT) calculations 
 Applications in materials for batteries and catalysts 
 Material properties prediction 
 Ab-initio calculations 
 Deep learning architectures 
 Graph neural networks 
 Energetic and electronic structures of defects in 2D materials 
 Sparse 

In [59]:

answer = qa_document_chain.run(
           input_document=" \n".join(list_dominan),
            question=(f""""
                 You are an expert in analyzing scientific papers.
                 Read the list carefully and 
                 create a new list of 10 research areas that will include all research areas specified in this list
                 """)
            )

In [60]:
print(answer)

1. Two-Dimensional Materials and Their Properties
2. Machine Learning Approaches in Materials Science
3. Defect Engineering and Properties in Crystalline and 2D Materials
4. Quantum Computing Applications in Materials Research
5. Energy Conversion and Storage Solutions: Photovoltaics and Batteries
6. Catalysts and Their Applications in Energy Systems
7. Electronic Structure and Band Gap Properties of Materials
8. Data-Driven Approaches for Material Design and Modification
9. Disorder and Its Effects on Electronic Properties in Materials
10. High Throughput Calculations and First-Principles Methods in Material Analysis


In [61]:
[w[2:] for w in answer.split("\n") if  len(w) and w[0].isdigit()]

[' Two-Dimensional Materials and Their Properties',
 ' Machine Learning Approaches in Materials Science',
 ' Defect Engineering and Properties in Crystalline and 2D Materials',
 ' Quantum Computing Applications in Materials Research',
 ' Energy Conversion and Storage Solutions: Photovoltaics and Batteries',
 ' Catalysts and Their Applications in Energy Systems',
 ' Electronic Structure and Band Gap Properties of Materials',
 ' Data-Driven Approaches for Material Design and Modification',
 ' Disorder and Its Effects on Electronic Properties in Materials',
 '. High Throughput Calculations and First-Principles Methods in Material Analysis']

In [62]:
list_dominan = []

first = True
for doc in documents:
    answer = qa_document_chain.run(
            input_document=doc.page_content,
            question=(f""""
                 You are an expert in analyzing scientific papers.
                 Read  a text carefully and answer of the question:
                 What are the five subject areas of research and the text study?
                 """)
            )
   #     first = False
   # else:
   #     answer = qa_document_chain.run(
   #         input_document=doc.page_content,
   #         question=(f""""
   #              You are an expert in analyzing scientific papers.
   #              Read a text carefully and answer of the question:
   #              What are the five subject areas of research and the text study?
   #              """)
   #         )
    if len(list_dominan) > 10:
        answer = qa_document_chain.run(
            input_document=" \n".join(list_dominan),
            question=(f""""
                 You are an expert in analyzing scientific papers.
                 Read the list carefully and 
                 create a new list of 10 research areas that will include all research areas specified in this list
                 """)
            )
    print(doc.metadata)
    print(answer)
    print()
    list_dominan += [w[2:] for w in answer.split("\n") if  len(w) and w[0].isdigit()]
    print(list_dominan)

{'source': './data/data_rag/s41524-023-01062-z.pdf', 'page': 0}
The relevant text mentions the following subject areas of research:

1. Two-dimensional materials
2. (Opto-) electronic devices
3. High technology applications
4. Properties of defects in 2D materials
5. Machine learning approaches for material property estimation

These areas can be inferred from the context of the text provided.

{'source': './data/data_rag/s41524-023-01062-z.pdf', 'page': 0}
I don't know.

{'source': './data/data_rag/s41524-023-01062-z.pdf', 'page': 1}
I don't know.

{'source': './data/data_rag/s41524-023-01062-z.pdf', 'page': 1}
I don't know.

{'source': './data/data_rag/s41524-023-01062-z.pdf', 'page': 2}
I don't know.

{'source': './data/data_rag/s41524-023-01062-z.pdf', 'page': 2}
I don't know.

{'source': './data/data_rag/s41524-023-01062-z.pdf', 'page': 3}
I don't know.

{'source': './data/data_rag/s41524-023-01062-z.pdf', 'page': 3}
I don't know.

{'source': './data/data_rag/s41524-023-01062-z.pd

{'source': './data/data_rag/s41699-023-00369-1.pdf', 'page': 6}
1. Intelligent Design and Modeling of Materials
2. Two-Dimensional Materials and Their Applications
3. (Opto-) Electronic Devices Utilizing 2D Materials
4. Defect Engineering in Two-Dimensional Materials
5. Quantum Point Defects in Two-Dimensional Materials
6. First-Principles Calculations for Defects in Solids
7. Modification of Physical Properties through Alloying and Heterostructures
8. Machine Learning Approaches in Material Property Estimation and Design
9. Data-Driven Understanding of Defect Properties in Materials
10. Catalyst Datasets and Community Challenges in Material Science

{'source': './data/data_rag/s41699-023-00369-1.pdf', 'page': 6}
1. Two-Dimensional Materials and Their Applications
2. (Opto-) Electronic Devices Utilizing 2D Materials
3. Defect Engineering in Two-Dimensional Materials
4. Quantum Point Defects in Two-Dimensional Materials
5. First-Principles Calculations for Defects in Solids
6. Modificat